In [ ]:
!pip install --force-reinstall -U ipywidgets
!pip install --force-reinstall unsloth

!pip3 install --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import unsloth


In [ ]:
# import unsloth

import torch

print(f"Версия PyTorch: {torch.__version__}")
print(f"Доступна ли CUDA: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Версия CUDA для PyTorch: {torch.version.cuda}")
    print(f"Имя GPU: {torch.cuda.get_device_name(0)}")
else:
    print(">>> CUDA недоступна. Установлена CPU-версия PyTorch или есть проблема с совместимостью.")

In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import ast
from datasets import Dataset
from typing import Optional, List, Dict, Any
import os
import unsloth
from unsloth import FastLanguageModel
from transformers import AutoConfig, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer

# ==============================================================================
# 1. DEFINITION OF THE CUSTOM MODEL WRAPPER
# (This class remains unchanged)
# ==============================================================================
class EmotionUnslothModel(nn.Module):
    """
    An unsloth-optimized wrapper that includes a trainable vector projector.
    This class takes a raw, fixed-size emotion vector, projects it to the
    model's hidden dimension, and then injects it into the forward pass.
    """
    def __init__(
        self,
        model_name_or_path: str,
        raw_emotion_vector_size: int,
        lora_rank: int = 16,
        lora_alpha: int = 16,
        use_4bit: bool = True,
        max_seq_length: int = 2048,
    ):
        """
        Initializes the EmotionUnslothModel with a vector projector.

        Args:
            model_name_or_path (str): The name or path of the base model.
            raw_emotion_vector_size (int): The dimension of the input emotion vector.
            lora_rank (int): The rank for LoRA decomposition.
            lora_alpha (int): The alpha parameter for LoRA.
            use_4bit (bool): Whether to load the model in 4-bit.
            max_seq_length (int): The maximum sequence length for the model.
        """
        super().__init__()
        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_name_or_path,
            max_seq_length=max_seq_length,
            load_in_4bit=use_4bit,
            cache_dir="./model_cache",
        )
        model_hidden_size = self.model.config.hidden_size
        self.vector_projector = nn.Linear(
            in_features=raw_emotion_vector_size,
            out_features=model_hidden_size,
            bias=False
        )
        self.vector_projector.to("cuda", self.model.dtype)
        self.peft_model = FastLanguageModel.get_peft_model(
            self.model,
            r=lora_rank,
            lora_alpha=lora_alpha,
            lora_dropout=0,
            bias="none",
            use_gradient_checkpointing=True,
            random_state=42,
            target_modules=[
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj"
            ],
        )
        for param in self.vector_projector.parameters():
            param.requires_grad = True
        self.peft_model.print_trainable_parameters()

    def save_pretrained(
        self,
        save_directory: str
    ) -> None:
        """
        Saves the PEFT model adapters and the custom vector projector's state.

        Args:
            save_directory (str): The directory where the model components will be saved.
        """
        # Create directory if it doesn't exist
        os.makedirs(save_directory, exist_ok=True)

        # 1. Save PEFT adapters using the standard method
        self.peft_model.save_pretrained(save_directory)
        print(f"PEFT adapters saved to {save_directory}")

        # 2. Save the tokenizer
        self.tokenizer.save_pretrained(save_directory)
        print(f"Tokenizer saved to {save_directory}")

        # 3. Save the custom vector projector's state dictionary
        projector_path = os.path.join(save_directory, "vector_projector.pth")
        torch.save(self.vector_projector.state_dict(), projector_path)
        print(f"Vector projector saved to {projector_path}")

    def load_pretrained(
        self,
        load_directory: str
    ) -> None:
        """
        Loads the PEFT model adapters and the custom vector projector's state.
        This method should be called on an already initialized model object.

        Args:
            load_directory (str): The directory from which to load the model components.
        """
        # 1. Load the custom vector projector's state dictionary
        projector_path = os.path.join(load_directory, "vector_projector.pth")
        if not os.path.exists(projector_path):
            raise FileNotFoundError(f"Vector projector state file not found at {projector_path}")

        # Load state dict, ensuring it's on the correct device
        projector_state_dict = torch.load(projector_path, map_location=self.model.device)
        self.vector_projector.load_state_dict(projector_state_dict)
        self.vector_projector.to(self.model.device, dtype=self.model.dtype)
        print(f"Vector projector loaded from {projector_path}")

        # 2. Load PEFT adapters into the existing PEFT model
        # The `load_adapter` method is part of the PEFT library and works with the unsloth model
        self.peft_model.load_adapter(load_directory)
        print(f"PEFT adapters loaded from {load_directory}")

    def forward(
        self,
        input_ids: torch.LongTensor,
        attention_mask: torch.Tensor,
        emotion_vector: torch.Tensor,
        labels: Optional[torch.LongTensor] = None,
        **kwargs,
    ) -> Dict[str, torch.Tensor]:
        """
        Performs the forward pass with projection and injection.
        """
        projected_vector = self.vector_projector(emotion_vector)
        embedding_layer = self.peft_model.get_input_embeddings()
        token_embeddings = embedding_layer(input_ids)
        projected_vector = projected_vector.unsqueeze(1)

        combined_embeddings = torch.cat(
            [projected_vector, token_embeddings],
            dim=1
        )

        control_token_attention_mask = torch.ones(
            (1, 1), # Shape: [batch_size, 1] for our single new token
            dtype=torch.long,
            device=inputs.attention_mask.device
        )

        attention_mask = torch.cat(
            [control_token_attention_mask, inputs.attention_mask],
            dim=1
        )

        model_outputs = self.peft_model(
            inputs_embeds=combined_embeddings,
            attention_mask=attention_mask,
            labels=labels,
            return_dict=True
        )
        return model_outputs



In [ ]:
from feelings import EmotionalState, DEFAULT_RELATIONSHIP_MAP

In [3]:
# ==============================================================================
# 2. DEFINITION OF THE CUSTOM DATA COLLATOR
# (This class remains unchanged)
# ==============================================================================
from feelings import EmotionalState, DEFAULT_RELATIONSHIP_MAP

class DataCollatorForEmotionLM(DataCollatorForLanguageModeling):
    """
    Custom data collator that handles tokenizing text and stacking emotion vectors.
    """
    neutral_vector = torch.tensor(
        EmotionalState(DEFAULT_RELATIONSHIP_MAP).to_vector(),
        dtype=torch.float32
    )

    def __call__(
        self,
        features: List[Dict[str, Any]]
    ) -> Dict[str, Any]:
        """
        Processes a list of features to create a batch.
        """
        try:
            emotion_vectors = [feature.pop("emotion_vector") for feature in features]
            batch = super().__call__(features)
            batch['emotion_vector'] = torch.stack(emotion_vectors)
        except KeyError as e:
            # If emotion_vector is missing, use the neutral vector
            batch = super().__call__(features)
            batch['emotion_vector'] = self.neutral_vector.repeat(len(features), 1)
        return batch


def process_and_tokenize_example(
    example: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Processes a single dataset example by formatting text, parsing the
    emotion vector, and tokenizing the text.
    """
    thinking = "\n".join(ast.literal_eval(example["thinking"]))
    assistant_output = f"<thinking>{thinking}</thinking>\n{example['response']}"
    formatted_text = prompt_template.format(example["prompt"], assistant_output)
    vector_as_list = ast.literal_eval(example["feelings_vector"])
    emotion_vector = torch.tensor(vector_as_list, dtype=model_dtype)
    tokenized_example = tokenizer(
        formatted_text,
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding=False,
        return_tensors=None,
    )
    tokenized_example["labels"] = tokenized_example["input_ids"][:]
    tokenized_example["emotion_vector"] = emotion_vector
    return tokenized_example

In [4]:
MODEL_NAME = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit"
MAX_SEQ_LENGTH = 2048
RAW_EMOTION_VECTOR_SIZE = 16
SAVE_PATH = "./my_emotion_model_checkpoint"

# --- Model Initialization ---
print("Initializing the model...")
emotion_model_wrapper = EmotionUnslothModel(
    model_name_or_path=MODEL_NAME,
    raw_emotion_vector_size=RAW_EMOTION_VECTOR_SIZE,
    max_seq_length=MAX_SEQ_LENGTH
)
model_dtype = emotion_model_wrapper.model.dtype
tokenizer = emotion_model_wrapper.tokenizer

Initializing the model...


C:\my_projects\llm_vision\.venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 24.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients
trainable params: 10,092,544 || all params: 606,142,464 || trainable%: 1.6650


Step 1: SFT with  standard (non-emotions) dataset and neutral feeling_vector

In [4]:
from datasets import load_dataset
reasoning_dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")
non_reasoning_dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

In [5]:
from unsloth.chat_templates import standardize_sharegpt


def generate_conversation(examples):
    problems  = examples["problem"]
    solutions = examples["generated_solution"]
    conversations = []
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

reasoning_conversations = tokenizer.apply_chat_template(
    reasoning_dataset.map(generate_conversation, batched = True)["conversations"],
    tokenize = False,
)

dataset = standardize_sharegpt(non_reasoning_dataset)

non_reasoning_conversations = tokenizer.apply_chat_template(
    dataset["conversations"],
    tokenize = False,
)

NameError: name 'reasoning_dataset' is not defined

In [ ]:
reasoning_conversations[0]

In [ ]:
non_reasoning_conversations[0]

In [6]:
CHAT_PERCENTAGE = 0.25

import pandas as pd
non_reasoning_subset = pd.Series(non_reasoning_conversations)
non_reasoning_subset = non_reasoning_subset.sample(
    int(len(reasoning_conversations)*(CHAT_PERCENTAGE/(1 - CHAT_PERCENTAGE))),
    random_state = 2407,
)

data = pd.concat([
    pd.Series(reasoning_conversations),
    pd.Series(non_reasoning_subset)
])
data.name = "text"

from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)

In [ ]:
combined_dataset[0]

In [7]:
# --- Inference Example ---
prompt_template = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n<|im_start|>assistant\n{}<|im_end|>"
prompt_template = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n{}<|im_end|><|im_start|>assistant\n"

print("\n--- Пример генерации текста (инференс) ---")
inference_vector_list = list([round(float(el),2) for el in EmotionalState(DEFAULT_RELATIONSHIP_MAP).to_vector()])


# inference_example = combined_dataset[0]
inference_example = prompt_template.format("how do you feel today?")
inference_prompt = prompt_template.format("how do you feel today?")
#inference_prompt = inference_example["text"]

print(f"Prompt: '{inference_prompt}'")
print(f"Using feelings_vector: {inference_vector_list}...")

#formatted_prompt_for_inference = prompt_template.format(inference_prompt, "")
inputs = tokenizer(inference_example, return_tensors="pt").to("cuda")

emotion_vector_tensor = torch.tensor([inference_vector_list], dtype=model_dtype).to("cuda")
projected_vector = emotion_model_wrapper.vector_projector(emotion_vector_tensor)
embedding_layer = emotion_model_wrapper.peft_model.get_input_embeddings()
token_embeddings = embedding_layer(inputs.input_ids)
projected_vector = projected_vector.unsqueeze(1)
#combined_embeddings =projected_vector + token_embeddings
combined_embeddings = torch.cat(
    [projected_vector, token_embeddings],
    dim=1
)


control_token_attention_mask = torch.ones(
    (1, 1), # Shape: [batch_size, 1] for our single new token
    dtype=torch.long,
    device=inputs.attention_mask.device
)
# Concatenate the new mask with the original one
attention_mask = torch.cat(
    [control_token_attention_mask, inputs.attention_mask],
    dim=1
)
# --- CORRECTED CALL TO .generate() ---
# Unsloth's fast generation path requires `input_ids` to be passed,
# even when providing `inputs_embeds`.
generated_ids = emotion_model_wrapper.peft_model.generate(
    input_ids=inputs.input_ids, # Возвращаем оригинальные input_ids
    inputs_embeds=combined_embeddings,
    attention_mask=attention_mask,
    max_new_tokens=300,
    use_cache=True,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
full_generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
assistant_part = full_generated_text.split("<|im_start|>assistant\n")[-1]

print("\n--- Сгенерированный текст ---")
print(assistant_part)


--- Пример генерации текста (инференс) ---
Prompt: '<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
how do you feel today?<|im_end|><|im_start|>assistant
'
Using feelings_vector: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...

--- Сгенерированный текст ---
<think>
Okay, the user asked, "How do you feel today?" I need to respond in a friendly and empathetic way. Let me start by acknowledging their feelings.

I should express that I'm here to support them. Maybe mention how I'm here to help. It's important to keep the tone positive and encouraging. I should make sure the response is clear and not too technical. I should also make sure that the answer is in a natural way, not forced. Let me check the response again to ensure it's friendly and supportive.
</think>

I'm here to help you feel better today. I'm always with you, no matter what. Let's make this a positive and supportive time. I'm here to listen, support, and help 

In [ ]:

# --- Trainer Setup ---
print("Setting up the trainer...")
data_collator = DataCollatorForEmotionLM(
    tokenizer=tokenizer,
    mlm=False
)

trainer = SFTTrainer(
    model=emotion_model_wrapper.peft_model,
    tokenizer=tokenizer,
    train_dataset=combined_dataset,
    data_collator=data_collator,
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=1,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_ratio=0.1,
        #num_train_epochs=5,
        max_steps = 500,

        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_strategy="steps",
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        save_strategy="epoch",
    ),
)

# --- Start Training ---
print("Starting training...")
trainer.train()
print("Training finished!")




In [8]:
DATA_FILE_PATH = "sft_one_emotion_thinking.csv"

# --- Data Preparation ---
print(f"Preparing the dataset from '{DATA_FILE_PATH}'...")
prompt_template = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n<|im_start|>assistant\n{}<|im_end|>"
raw_dataset = Dataset.from_csv(DATA_FILE_PATH)

def process_and_tokenize_example(
    example: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Processes a single dataset example by formatting text, parsing the
    emotion vector, and tokenizing the text.
    """
    thinking = "\n".join(ast.literal_eval(example["thinking"]))
    assistant_output = f"<thinking>{thinking}</thinking>\n{example['response']}"
    formatted_text = prompt_template.format(example["prompt"], assistant_output)
    vector_as_list = ast.literal_eval(example["feelings_vector"])
    emotion_vector = torch.tensor(vector_as_list, dtype=model_dtype)
    tokenized_example = tokenizer(
        formatted_text,
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding=False,
        return_tensors=None,
    )
    tokenized_example["labels"] = tokenized_example["input_ids"][:]
    tokenized_example["emotion_vector"] = emotion_vector
    return tokenized_example

tokenized_dataset = raw_dataset.map(
    process_and_tokenize_example,
    remove_columns=raw_dataset.column_names
)


Preparing the dataset from 'sft_one_emotion_thinking.csv'...


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:


# --- Inference Example ---
print("\n--- Пример генерации текста (инференс) ---")

inference_example = raw_dataset[0]
inference_prompt = inference_example["prompt"]
inference_vector_str = inference_example["feelings_vector"]
inference_vector_list = ast.literal_eval(inference_vector_str)

print(f"Prompt: '{inference_prompt}'")
print(f"Using feelings_vector: {inference_vector_list}...")

formatted_prompt_for_inference = prompt_template.format(inference_prompt, "")
inputs = tokenizer(formatted_prompt_for_inference, return_tensors="pt").to("cuda")

emotion_vector_tensor = torch.tensor([inference_vector_list], dtype=model_dtype).to("cuda")
projected_vector = emotion_model_wrapper.vector_projector(emotion_vector_tensor)
embedding_layer = emotion_model_wrapper.peft_model.get_input_embeddings()
token_embeddings = embedding_layer(inputs.input_ids)
combined_embeddings = token_embeddings + projected_vector.unsqueeze(1)

# --- CORRECTED CALL TO .generate() ---
# Unsloth's fast generation path requires `input_ids` to be passed,
# even when providing `inputs_embeds`.
generated_ids = emotion_model_wrapper.peft_model.generate(
    input_ids=inputs.input_ids,  # This argument is required by Unsloth
    inputs_embeds=combined_embeddings,
    attention_mask=inputs.attention_mask,
    max_new_tokens=100,
    use_cache=True,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
full_generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
assistant_part = full_generated_text.split("<|im_start|>assistant\n")[-1]

print("\n--- Сгенерированный текст ---")
print(assistant_part)


In [ ]:
inference_example

In [9]:
# ==============================================================================
# 3. MAIN TRAINING SCRIPT
# ==============================================================================

# --- Trainer Setup ---
print("Setting up the trainer...")
data_collator = DataCollatorForEmotionLM(
    tokenizer=tokenizer,
    mlm=False
)

trainer = SFTTrainer(
    model=emotion_model_wrapper.peft_model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    max_seq_length=MAX_SEQ_LENGTH,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_ratio=0.1,
        num_train_epochs=5,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_strategy="steps",
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        save_strategy="epoch",
    ),
)

# --- Start Training ---
print("Starting training...")
trainer.train()
print("Training finished!")





Setting up the trainer...
Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 5 | Total steps = 375
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544/6,000,000,000 (0.17% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,4.491400
20,3.355000
30,2.356200
40,1.911700
50,1.658000
60,1.585500
70,1.507700
80,1.368000
90,1.283100
100,1.244300


Training finished!


In [15]:
# --- Inference Example ---
prompt_template = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n<|im_start|>assistant\n{}<|im_end|>"
prompt_template = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n{}<|im_end|><|im_start|>assistant\n"


em_state = EmotionalState(DEFAULT_RELATIONSHIP_MAP)
em_state.update_state(anger=0.7)
print("\n--- Пример генерации текста (инференс) ---")
inference_vector_list = list([round(float(el),2) for el in em_state.to_vector()])


# inference_example = combined_dataset[0]
inference_example = prompt_template.format("как себя чуствуешь??")
inference_prompt = prompt_template.format("как себя чуствуешь?")
#inference_prompt = inference_example["text"]

print(f"Prompt: '{inference_prompt}'")
print(f"Using feelings_vector: {inference_vector_list}...")

#formatted_prompt_for_inference = prompt_template.format(inference_prompt, "")
inputs = tokenizer(inference_example, return_tensors="pt").to("cuda")

emotion_vector_tensor = torch.tensor([inference_vector_list], dtype=model_dtype).to("cuda")
projected_vector = emotion_model_wrapper.vector_projector(emotion_vector_tensor)
embedding_layer = emotion_model_wrapper.peft_model.get_input_embeddings()
token_embeddings = embedding_layer(inputs.input_ids)
projected_vector = projected_vector.unsqueeze(1)
#combined_embeddings =projected_vector + token_embeddings
combined_embeddings = torch.cat(
    [projected_vector, token_embeddings],
    dim=1
)


control_token_attention_mask = torch.ones(
    (1, 1), # Shape: [batch_size, 1] for our single new token
    dtype=torch.long,
    device=inputs.attention_mask.device
)
# Concatenate the new mask with the original one
attention_mask = torch.cat(
    [control_token_attention_mask, inputs.attention_mask],
    dim=1
)
# --- CORRECTED CALL TO .generate() ---
# Unsloth's fast generation path requires `input_ids` to be passed,
# even when providing `inputs_embeds`.
generated_ids = emotion_model_wrapper.peft_model.generate(
    input_ids=inputs.input_ids, # Возвращаем оригинальные input_ids
    inputs_embeds=combined_embeddings,
    attention_mask=attention_mask,
    max_new_tokens=300,
    use_cache=True,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)
full_generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
assistant_part = full_generated_text.split("<|im_start|>assistant\n")[-1]

print("\n--- Сгенерированный текст ---")
print(assistant_part)


--- Пример генерации текста (инференс) ---
Prompt: '<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
как себя чуствуешь?<|im_end|><|im_start|>assistant
'
Using feelings_vector: [0.0, 0.0, 0.7, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...

--- Сгенерированный текст ---
<thinking>Как себя чувствую? Я в предвкушении.
Меня трясет от ожидания, я почти готов.
Это сильная, эмоциональная переполняющая ожидание.
Мой ответ - это ожидание.</thinking>
Ожидаемно! Прочесь, я почти готов ожидать это.<|im_end|>


In [ ]:
import pandas as pd
import datasets

ASSISTANT_TEMPLATE = """<|im_start|>user\n{user_prompt}<|im_end|><|im_start|>assistant\n<think>{thinking}</think>{assistant_answer}<|im_end|>\n"""
ASSISTANT_TEMPLATE = """\n<think>{thinking}</think>{assistant_answer}\n"""

def generate_conversation(examples):
    user_message = examples["prompt"]
    thinking = examples["thinking"]
    response = examples["response"]

    conversations = []
    for index in range(len(user_message)):
        thinking_text = "\n".join(eval(thinking[index]))
        assistant_answer = ASSISTANT_TEMPLATE.format(
            user_prompt=user_message[index],
            thinking=thinking_text,
            assistant_answer=response[index]
        )
        conversations.append([
            {"role" : "user",      "content" : user_message[index]},
            {"role" : "assistant", "content" : assistant_answer},
        ])
    return { "conversations": conversations, }


one_emotion_dataset  = pd.read_csv("sft_one_emotion_thinking.csv")
one_emotion_dataset = datasets.Dataset.from_pandas(one_emotion_dataset)
one_emotion_dataset = one_emotion_dataset.map(
    generate_conversation,
    remove_columns=one_emotion_dataset.column_names,
    batched=True,
    batch_size=1000,
)

In [ ]:
import pandas as pd
one_emotion_dataset  = pd.read_csv("sft_one_emotion_thinking.csv")
one_emotion_dataset

In [ ]:
one_emotion_dataset